# Natural Language Processing

### 1. Install Required Libraries
Ensure that you have the necessary libraries installed. You'll need scikit-learn, numpy, and pandas, which can be installed via pip:

In [2]:
pip install scikit-learn numpy pandas

Note: you may need to restart the kernel to use updated packages.


### 2. Import Libraries
Import the required libraries in your Python script or Jupyter Notebook:

In [3]:
import numpy as np
import pandas as pd
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.svm import LinearSVC


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.metrics import accuracy_score

### 3. Load and Preprocess Data
Load your labeled dataset into a pandas DataFrame. Preprocess the text data by cleaning and preparing it for analysis, including steps like removing punctuation, stopwords, and performing tokenization. Ensure that your DataFrame has two columns: "text" (containing the text data) and "label" (containing the corresponding labels).

In [5]:
df_original = pd.read_csv('/Users/xiaoa1/VSCode/ds/_notebooks/data/train_40k.csv')

df_val = pd.read_csv('/Users/xiaoa1/VSCode/ds/_notebooks/data/val_10k.csv')

columns = ['Text', 'Score']

df = shuffle(df_original[columns])

df.Score.value_counts()

p = re.compile(r'[^\w\s]+')

df['Text'] = [p.sub('', x) for x in df['Text'].tolist()]

df.apply(lambda x: x.astype(str).str.lower())

,Text,Score
29821,i am a devout amazoncom customer as well as bu...,1.0
39902,im very particular about what i feed my dogs i...,5.0
26709,this cushion is primarily used when dog is out...,4.0
376,my stepbrother had one of these on his keyring...,4.0
32764,i have used real maple syrup for years when my...,5.0
...,...,...
2716,although this inflatable seems wellmade and du...,3.0
32173,im on my second day of using this body wash an...,5.0
38912,i bought this for my nieces first birthday it ...,4.0
32844,heard about this product and was getting tired...,4.0


### 4. Split the Dataset
Split your dataset into training and testing sets using train_test_split from scikit-learn. This ensures that you have data for training the model as well as evaluating its performance:

In [7]:
x,y = df.Text, df.Score
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)

### 5. Feature Extraction
Convert the text data into numerical feature vectors that can be used by machine learning algorithms. Use scikit-learn's CountVectorizer to convert text into a matrix of token counts:

In [8]:
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2), stop_words='english', sublinear_tf=True)),
                     ('chi', SelectKBest(chi2, k=10000)),
                     ('clf', LinearSVC(C=1.0, penalty='l1',max_iter=3000, dual=False))
                    ])

### 6. Model Training
Choose a classification algorithm, such as Support Vector Machines (SVM), and train the model using the training data:

In [9]:
model = pipeline.fit(train_x, train_y)

### 7. Model Evaluation
Use the trained model to predict the labels for the test data, and evaluate its performance using accuracy or other suitable metrics:

In [10]:
print('accuracy score: '+ str(model.score(test_x, test_y)))

accuracy score: 0.650875


### 8. Prediction
Once your model is trained and evaluated, you can use it to classify new, unseen text data by transforming the text into feature vectors using the CountVectorizer and then applying the trained model's predict method.

In [14]:
print(model.predict(['price is good but tasted a bit stale']))

[4.]
